<a href="https://colab.research.google.com/github/jeffheaton/app_generative_ai/blob/main/t81_559_class_03_4_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T81-559: Applications of Generative Artificial Intelligence
**Module 3: Large Language Models**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module 3 Material

* Part 3.1: Foundation Models [[Video]](https://www.youtube.com/watch?v=Gb0tk5qq1fA) [[Notebook]](t81_559_class_03_1_llm.ipynb)
* Part 3.2: Text Generation [[Video]](https://www.youtube.com/watch?v=lB97Lqt7q58) [[Notebook]](t81_559_class_03_2_text_gen.ipynb)
* Part 3.3: Text Summarization [[Video]](https://www.youtube.com/watch?v=3MoIUXE2eEU) [[Notebook]](t81_559_class_03_3_text_summary.ipynb)
* **Part 3.4: Text Classification** [[Video]](https://www.youtube.com/watch?v=2VpOwFIGmA8) [[Notebook]](t81_559_class_03_4_classification.ipynb)
* Part 3.5: LLM Writes a Book [[Video]](https://www.youtube.com/watch?v=iU40Rttlb_Q) [[Notebook]](t81_559_class_03_5_book.ipynb)


# Google CoLab Instructions

The following code ensures that Google CoLab is running and maps Google Drive if needed.

In [2]:
import os

try:
    from google.colab import drive, userdata
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# OpenAI Secrets

# OpenAI Secrets
if COLAB:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Install needed libraries in CoLab
if COLAB:
    !pip install langchain langchain_openai

Note: using Google CoLab
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 kB 3.1 MB/s eta 0:00:00


# 3.4: Text Classification


Large Language Models (LLMs) have revolutionized the field of text classification by offering a more flexible and efficient approach compared to traditional machine learning methods. Traditionally, text classification required extensive labeled datasets for training machine learning models. This process was time-consuming and resource-intensive, as it involved manually annotating a large number of examples for each category.

In contrast, LLMs, such as those based on the GPT architecture, can perform text classification using a technique known as "zero-shot" learning. This innovative method allows the model to classify text without needing any labeled examples beforehand. Instead of relying on a dataset of annotated examples, we provide the LLM with a natural language description of the classification task. The model then uses its vast knowledge, acquired from training on diverse and extensive text data, to understand and perform the classification based on the given description.

This zero-shot capability significantly reduces the time and effort required to set up a text classification system. It also enables rapid adaptation to new and evolving classification tasks, making LLMs a powerful tool for various applications, from sentiment analysis and topic categorization to spam detection and content moderation.

The following code gets sample emails that a professor might get from faculty, students and family members. We will write code to classify these emails.





In [3]:
import requests

def get_email(i):
    if i<1 or i>10:
        raise Exception("Invalid email number")

    # URL to download
    url = f"https://data.heatonresearch.com/wustl/CABI/genai-langchain/emails/email_{i}.txt"

    # Perform a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Convert the content of the response to a string
        content = response.text
        return content
    else:
        raise Exception("Failed to retrieve the content")


For example, the following displays email \#1.

In [4]:
print(get_email(1))

Dear Professor Lawson,

I'm Alex Chen, leader of team Nova in the Data Science Challenge (Spring 2019).

I am seeking to pursue a PhD in Computer Science, and I am hoping you could provide a recommendation letter for my applications.

This term has been unexpectedly demanding, leading to delays in preparing my applications. I acknowledge the timing is not ideal, but I am committed to completing my applications diligently.

Your guidance in CSC 402 Advanced Machine Learning and your support in my research project have been invaluable. I was particularly drawn to this field, prompting me to enroll in your course despite it not contributing to my major credits. The course proved to be highly beneficial, enriching my understanding through online lectures and practical assignments. I excelled in the course, securing an A+ and leading my team to the top position in the semester's Data Science Challenge.

In the previous term (Fall 2019), I encountered a challenging issue in my project on "Au

Now create a program that loops over all 10 and classifies each email as one of the following.

* spam - For marketing emails trying to sell something
* faculty - For faculty annoucements and requests
* help - For students requesting help on an assignment
* lor - For students requesting a letter of recommendation

If the email fits into none of these, then classify it as "other". If the email is a request for help on an assignment, determine the assignment number the student is asking about.

The following is sample is how we will output.

```
Email #1 is: spam
Email #2 is: other
Email #3 is: ...

```


In [5]:
from langchain_openai import ChatOpenAI

MODEL = 'gpt-5'
TEMPERATURE = 0.0

# Initialize the OpenAI LLM with your API key
llm = ChatOpenAI(
    model=MODEL,
    temperature=TEMPERATURE,
    n=1
)

This code demonstrates the use of a large language model (LLM) for classifying and extracting information from emails using the Langchain framework. The process starts by importing necessary classes and modules from Langchain, including HumanMessage, SystemMessage, AIMessage, ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate, PromptTemplate, LLMChain, and SimpleSequentialChain.

Two prompt templates are defined using PromptTemplate. The first template, email_prompt, takes an email as input and asks the model to classify it into one of five categories: spam, faculty, help, lor (letter of recommendation), or other. The second template, help_prompt, is designed to extract the assignment number from emails classified as "help," where students are requesting assistance with an assignment.

The code then creates two LLM chains, chain_email and chain_help, by combining the respective prompts with the LLM. These chains are used to process the emails.

The main loop iterates over a range of 10 emails (from 1 to 10). For each email, it retrieves the email content using the get_email(i) function. The email content is then classified using chain_email.invoke(email), which returns the classification as a string. If the classification is "help," the email is further processed with chain_help.invoke(email) to extract the assignment number. The extracted information is printed accordingly: either indicating the assignment number for help-related emails or the classification for other types of emails.

In [6]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

email_prompt = PromptTemplate( input_variables = ['email'], template = """
Classify the following email as either:
* spam - For marketing emails trying to sell something
* faculty - For faculty annoucements and requests
* help - For students requesting help on an assignment
* lor - For students requesting a letter of recommendation
* other - If it does not fit into any of these.
Here is the email. Return code, such as spam. Return nothing else, do not explain your choice.
Make sure that if the email does not fit into one of the categories that you classify it as other.
Here is the email:

{email}""")

help_prompt = PromptTemplate( input_variables = ['email'], template = """
You are given an email where a student is asking about an assignment. Return the assignment number
that they are asking about. If you cannot tell return a ?. Return only the assignment number as
an integer, do not explain.
Here is the email:

{email}""")

chain_email = email_prompt | llm
chain_help = help_prompt | llm

for i in range(1,11):
    email = get_email(i)
    classification = chain_email.invoke(email).content.strip()
    if classification == 'help':
        assignment = chain_help.invoke(email).content.strip()
        print(f"Email #{i} is a question about assignment #{assignment}")
    else:
        print(f"Email #{i} is: {classification}")



Email #1 is: lor
Email #2 is a question about assignment #7
Email #3 is: faculty
Email #4 is: lor
Email #5 is: spam
Email #6 is: other
Email #7 is a question about assignment #3
Email #8 is: faculty
Email #9 is: spam
Email #10 is: other


## Classify Names

Spell checkers can be tricky to code. With the large diversity of names around the world, sometimes people's names are flagged as typos. There is a campaign named [#iamnotatypo](https://www.iamnotatypo.org/) that attempts to build a list of people's names; however, maybe AI can help. As a human, usually I can determine if words are a name just from context. Consider the following paragraph where many of the names would be flagged by MS-Word as typos. Can GenAI find all the names in this paragraph? Just to make it more like text a human would generate, I "forgot" to capitalize a few of the names.

Make use of LangChain to extract just the names, as a Python list.

In [7]:
SAMPLE = """
Esmae and Zarah have been friends since college. After graduation they interviewed with priti
for a job at the technotryne corporation, whose flagship product is named Futuricore, which
was developed by a programmer named María José García Martínez. After a successful interview
they met with their friends Rafe and Ayda to celebrate good times. Their first day on the job,
they met three other employees, Ruaridh, Eesa, and Otillie. Later they were joined by
Matei Smith."""

Now we define an LLM to parse through the above paragraph.

In [8]:
from langchain_openai import ChatOpenAI

MODEL = 'gpt-4o-mini'
TEMPERATURE = 0.0

# Initialize the OpenAI LLM with your API key
llm = ChatOpenAI(
    model=MODEL,
    temperature=TEMPERATURE,
    n=1
)

Finally, we use a one-shot prompt to classify and extract these names.

In [9]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    template="""
    Find all human names in the following text. Extract complete name, return only names.\n
    Output each name on a separate line.\n
    {subject}.\n""",
    input_variables=["subject"]
)

chain = prompt | llm

print(chain.invoke({"subject": SAMPLE}).content)

Esmae  
Zarah  
Priti  
María José García Martínez  
Rafe  
Ayda  
Ruaridh  
Eesa  
Otillie  
Matei Smith  
